In [59]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import time
import warnings

In [60]:
def number_of_days_in_year(year):
    y1 = datetime.date(year, 1,1)
    y2 = datetime.date(year+1, 1,1)
    return (y2-y1).days # days from datetime.timedelta

def generate_days(year):
    current_day = datetime.date(year, 1, 1)
    last_day = datetime.date(year, 12, 31)
    day_increment = datetime.timedelta(days=1)
    while current_day <= last_day:
        yield current_day
        current_day += day_increment

def calculate_day(**kwargs):
    accounts = kwargs.get('accounts', {})
    debts = kwargs.get('debts', {})
    income = kwargs.get('income', {})
    payments = kwargs.get('payments', {})
    rates = kwargs.get('rates',{})

    for key in accounts:
        accounts[key] = accounts[key]*(1+rates.get(key, 1.0)) +\
            income.get(key, 0.0) - payments.get(key, 0.0)
    for key in debts:
         debts[key] = debts[key]*(1+rates.get(key, 1.0)) -\
            payments.get(key, 0.0)
    return accounts, debts

def is_payday(weekday=5):
    payweek = False # assume first friday is not payday
    def is_pay_weekday(weekdate):
        return weekdate.isoweekday() == weekday
    while True:
        date = yield # fed this value
        yield payweek and is_pay_weekday(date)
        payweek = payweek if not is_pay_weekday(date) else not payweek

In [61]:
class Loan:
    def __init__(self, name: str, principle: float,
                apr: float, length: int, units: str='years'):
        """ class for managing a loan """
        self.name = name
        self.principle = principle
        self.apr = apr
        self.length = length
        if units in ['y','year','years']:
            self.units = 'years'
        elif units in ['m','month','months']:
            self.units = 'months'
        else:
            raise ValueError("bad value passed to 'units'. Accepts 'years' or 'months'.")

        rate = apr/12 # for monthly payments
        # if length is in years convert to months for amortization calculation
        length = self.length * 12 if self.units == 'years' else self.length
        self.minimum = self.principle *\
            rate * ((1+rate)**length) /\
            ( ((1+rate)**length) + 1 )
        self._origination = self.principle
    
    def __repr__(self):
        """ returns the string representation of the class """
        return f"Loan(name={self.name}, origination=${self.origination:.2f}, length={self.length} {self.units})"

    def get_origination(self):
        """ get the origination value of the loan """
        return self._origination
    # disallow changes to the origination value
    origination = property(fget=get_origination)

    def make_payment(self, value=None) -> float:
        """ method to make payments on the loan, returning any leftover """
        value = value if value else self.minimum
        if value < self.minimum: warnings.warn('Failed to make the minimum payment on {self.name}')
        if self.principle < value:
            amount = self.principle
            leftover = value-self.principle
        else: # no leftover remaining
            amount = value
            leftover = 0.0
        self.principle -= amount
        return round(leftover, 2)

l = Loan(name='House', principle=175000, apr=.0325, length=30)
print(l.origination)
print(l)
print(l.principle)
l.make_payment()
print(l.principle)

175000
Loan(name=House, origination=$175000.00, length=30 years)
175000
174655.97601520194


In [62]:
test_loan = Loan(name='Test', principle=150000, apr=.04, length=30)
should_be = 954.83
print(test_loan.minimum)
counter = 0
while test_loan.principle > 0:
    test_loan.make_payment()
    counter += 1
    print(counter, test_loan.principle)
print(30*12, 'Done')

384.0847965370952
1 149615.9152034629
2 149231.83040692582
3 148847.74561038872
4 148463.66081385163
5 148079.57601731454
6 147695.49122077745
7 147311.40642424035
8 146927.32162770326
9 146543.23683116617
10 146159.15203462908
11 145775.067238092
12 145390.9824415549
13 145006.8976450178
14 144622.8128484807
15 144238.72805194362
16 143854.64325540652
17 143470.55845886943
18 143086.47366233234
19 142702.38886579525
20 142318.30406925816
21 141934.21927272106
22 141550.13447618397
23 141166.04967964688
24 140781.9648831098
25 140397.8800865727
26 140013.7952900356
27 139629.7104934985
28 139245.62569696142
29 138861.54090042433
30 138477.45610388723
31 138093.37130735014
32 137709.28651081305
33 137325.20171427596
34 136941.11691773887
35 136557.03212120177
36 136172.94732466468
37 135788.8625281276
38 135404.7777315905
39 135020.6929350534
40 134636.6081385163
41 134252.52334197922
42 133868.43854544213
43 133484.35374890504
44 133100.26895236794
45 132716.18415583085
46 132332.09935

In [63]:
?warnings.warn

Signature: warnings.warn(message, category=None, stacklevel=1, source=None)
Docstring: Issue a warning, or maybe ignore it or raise an exception.
Type:      builtin_function_or_method


In [64]:
start = time.time()
accounts = {
    'main':     1000.00,
}
debts = {
    'house':    175000.00,
}
rates = {
    'main':     0.01,
    'house':    0.0325,
}
pay = {
    'main':     2350.00,
}
expense = {
    'main':     1340.00,
    'house':    1000.00,
}
payday_gen = is_payday()
for year in range(2020, 2080):
    num_days = number_of_days_in_year(year)
    year_rates = {k:rates[k]/num_days for k in rates}
    # TODO: handle leap years wrt interest rates
    for day in generate_days(year):
        next(payday_gen) # prepare the generator
        payday_today = payday_gen.send(day) # update with current day
        bills_today = day.day == 1
        income = pay if payday_today else {}
        payments = {} if not bills_today else expense
        accounts, debts = calculate_day(accounts=accounts,
                                        debts=debts,
                                        rates=year_rates,
                                        income=income,
                                        payments=payments)
print(accounts)
print(debts)
end = time.time()
end-start

{'main': 3718492.707530984}
{'house': -998891.8344628954}


0.09313178062438965